#**Phương pháp 1: Cách lấy dữ liệu bằng thư viện VNQuant**

VNQuant là một thư viện được tạo ra bởi tác giả Phạm Đình Khánh, giúp mọi người truy cập và phân tích dữ liệu thị trường tài chính của Việt Nam. Thư viện này cung cấp các công cụ và chức năng để tải xuống dữ liệu về báo cáo tài chính và các chỉ số cơ bản của doanh nghiệp, đồng thời hỗ trợ phân tích kỹ thuật bằng cách cung cấp các biểu đồ nến, chỉ báo kỹ thuật và nhiều tính năng khác.

Dưới đây là một số ưu điểm và hạn chế của VNQuant:

**Ưu điểm:**

- VNQuant cho phép người dùng tải xuống dữ liệu liên quan đến báo cáo tài chính của các doanh nghiệp.

- VNQuant cung cấp dữ liệu về VNINDEX với cột volume đã được điều chỉnh, giúp người dùng phân tích thị trường một cách chính xác hơn.

- Thích hợp cho việc phân tích kỹ thuật: Thư viện này cung cấp các công cụ và tính năng thích hợp cho việc phân tích kỹ thuật, bao gồm biểu đồ nến, các chỉ báo kỹ thuật, và các tính năng khác giúp người dùng hiểu rõ hơn về xu hướng thị trường và điểm vào/ra khỏi thị trường.

**Hạn chế:**

- Phụ thuộc vào cơ sở dữ liệu và API của Vndirect: VNQuant phụ thuộc vào cơ sở dữ liệu và API của Vndirect để tải xuống dữ liệu, do đó nếu có bất kỳ thay đổi nào trong cơ sở dữ liệu hoặc gián đoạn dịch vụ từ Vndirect có thể ảnh hưởng đến việc tải dữ liệu của VNQuant.
- VNQuant không có tính năng tích hợp thông báo qua các kênh như Telegram và Slack khiến cho việc tự động hóa các quy trình giao dịch bị giảm.
- VNQuant không hỗ trợ dữ liệu từ các sàn giao dịch quốc tế khác nhau hoặc các loại sản phẩm khác như tiền điện tử, hàng hóa, chỉ số.
- VNQuant bị hạn chế về phạm vi dữ liệu lịch sử giá, không lấy được dữ liệu theo giờ hoặc phút

[Link bài viết chi tiết của QM Capital](https://www.qmcapital.vn/bai-viet/kien-thuc/kien-thuc-chung/cach-lay-du-lieu-bang-thu-vien-vnquant)

*Trên đây là một số ưu, nhược điểm của thư viện VNQuant mà QM Capital đã tổng hợp, hẹn mọi người trong phương pháp lấy dữ liệu bằng thư viện [Vnstock](https://www.qmcapital.vn/bai-viet/kien-thuc/kien-thuc-chung/cach-lay-du-lieu-bang-thu-vien-vnstock).*

#**Download dữ liệu lịch sử giá**

#**Phần I. Cài đặt thư viện**

In [ ]:
# Cài đặt thư viện `httpx` để hỗ trợ việc gửi yêu cầu HTTP.
%pip install httpx

In [ ]:
# Sao chép mã nguồn từ kho `vnquant` trên GitHub về máy của bạn để sử dụng các công cụ và dữ liệu cho phân tích.
!git clone https://github.com/phamdinhkhanh/vnquant.git
%cd vnquant
!pip install .

In [ ]:
# Cài đặt thư viện `vnquant` trực tiếp từ mã nguồn GitHub để sử dụng các tính năng
!pip install git+https://github.com/phamdinhkhanh/vnquant.git

In [ ]:
# Cài đặt thư viện `vnquant` từ PyPI, hỗ trợ phân tích dữ liệu chứng khoán Việt Nam.
!pip install vnquant

In [ ]:
%cd ..
!del -rf vnquant

# **Phần II. Download dữ liệu lịch sử giá**

In [ ]:
# Nhập mô-đun dữ liệu từ thư viện `vnquant` để tải và xử lý dữ liệu chứng khoán.
import vnquant.data as dt

# Tải và chuẩn bị dữ liệu chứng khoán cho mã 'MBB' từ `2013-01-01` đến `2022-10-01`, hiển thị 5 hàng đầu tiên của bộ dữ liệu.
loader = dt.DataLoader(
    symbols="MBB", start="2013-01-01", end="2022-10-01", minimal=True, data_source="VND"
)
data = loader.download()
data = data.stack()
data = data.reset_index()
data = data.set_index("date")
data.head()

[2024-12-11 19:34:50] - Assitant - INFO      - download - line 32 - Start downloading data symbols ['MBB'] from VNDIRECT, start: 01/01/2013, end: 01/10/2022!


API_VNDIRECT:https://finfo-api.vndirect.com.vn/v4/stock_prices/
{'timestamp': '2024-12-11T12:34:51.848+00:00', 'status': 404, 'error': 'Not Found', 'path': '/v4/stock_prices/'}


KeyError: 'data1'

# **Phần III. Trực quan hóa dữ liệu**

In [ ]:
# Tạo biểu đồ Candlestick và Volume cho mã 'MBB' sử dụng `plotly`
import plotly.subplots as ms
import plotly.graph_objects as go

fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Candlestick(x=data.index,
                             open=data['open'],
                             high=data['high'],
                             low=data['low'],
                             close=data['close'],
                             name='Candlestick'),
              row=1, col=1)

# Thêm biểu đồ cột thể hiện khối lượng giao dịch
fig.add_trace(go.Bar(x=data.index, y=data['volume'], name='Volume'),
              row=2, col=1)

# Cập nhật bố cục của biểu đồ, bao gồm tiêu đề, tiêu đề trục y, ẩn thanh trượt x, khoảng cách giữa các cột, và đặt phạm vi thời gian cho trục x.
fig.update_layout(title='MBB CandleStick Chart',
                  yaxis_title='MBB',
                  xaxis_rangeslider_visible=False,
                  bargap=0,
                  xaxis_range=['2018-01-01', '2023-12-10'])

# Hiển thị biểu đồ
fig.show()


# **Download các dữ liệu tài chính khác**

In [ ]:
# Dữ liệu các chỉ số tài chính cơ bản
import vnquant.data as dt
loader = dt.FinanceLoader('TCB', '2018-06-02','2022-10-30', minimal=True)
data_basic = loader.get_basic_index()
data_basic.head()